In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210505_07_54_14_000_T4CaxysrQ7W21Wgl1wZWBCtdRjE3_F_3264_2448.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210518_06_25_57_000_6SLj9T2MBmhTqitpNcjcjOVAYyG2_F_3000_4000.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210512_09_41_56_000_ovRKxW0QbBVfwN9RW4teROP1YgG3_F_4160_3120.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210512_13_32_08_000_vVGqfiibxjRMSkSyttjdIlsHrj92_F_3000_4000.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210517_13_00_53_000_QTHwLcCLJSgc8mdgtvUWq1eiMT52_F_3264_2448.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210522_00_57_36_000_z2umdg3LdTXQFkhWMsDipg7Sys22_T_4160_3120.jpg
/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/Tempo Traveller/20210521_09_35_46_000_3i9IwxEEa6UEWq4gEOQ2NQnD0uz2_F

In [14]:
import torch
import torch.nn as nn
from torch.optim.adam import Adam
from torch.optim.sgd import SGD
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ExponentialLR, LinearLR, StepLR

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from dataclasses import dataclass

import numpy as np
import pandas as pd
import random
import tqdm
import os

import matplotlib.pyplot as plt
import matplotlib
import typing as tp

In [15]:
seed: int = 0
device: str = 'cpu'
torch.cuda.is_available()

False

In [16]:
@dataclass
class Config:
    # Данные
    batch_size: int = 64
    do_shuffle_train: bool = True
    img_size: int = 224
    ratio_train_val_test: tuple[float, float, float] = (0.8, 0.1, 0.1)

    # Модель
    hidden_dim: int = 512
    p_dropout: float = 0.3
    n_alpha: float = 0.3

    # Обучение
    n_epochs: int = 100
    lr: float = 1e-5
    scheduler_type: tp.Literal["exp", "linear", "step"] | None = None
    optimizer_type: tp.Literal["sgd", "adam"] = "sgd"


def enable_determinism():
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.use_deterministic_algorithms(True)

def fix_seeds(seed: int):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.mps.manual_seed(seed)

config = Config()
enable_determinism()
fix_seeds(seed)

In [17]:
class GetDataset(Dataset):
    def __init__(self, path_folder, image_size):
        self.path_folder = path_folder
        self.paths = []
        self.labels = []
        self.image_size = image_size
        self.encoder = self.settings()
        self.transforms = A.Compose([
            A.Resize(self.image_size, self.image_size),
            A.Rotate(limit=45, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomCrop(width=self.image_size, height=self.image_size, p=0.5),
            A.ToFloat(max_value=255),
            ToTensorV2(),
        ])

    def __getitem__(self, index) -> tuple[torch.Tensor, list]:
        img_numpy = cv2.imread(self.paths[index])
        label = self.labels[index]
        img_tensor = self.transforms(image=img_numpy)
        return img_tensor['image'], torch.tensor(self.encoder.transform([label])[0])

    def show_images(self, nrows: int = 3, ncols: int = 3):
        choice: np.ndarray = np.random.randint(low=0, high=self.__len__(), size=(nrows*ncols,))
        fig: matplotlib.figure.Figure
        ax: np.ndarray
        fig, ax = plt.subplots(nrows, ncols, figsize=[28, 28])
        ind_img: int = 0
        for row in range(nrows):
            for col in range(ncols):
                image, label = self.__getitem__(choice[ind_img])
                ax[row, col].imshow(image.permute((1, 2, 0)))
                ax[row, col].set_title(self.encoder.inverse_transform([label.item()])[0], fontsize=28)
                ind_img += 1
        plt.tight_layout()
        plt.show()

    def __len__(self):
        return len(self.paths)
        
    def list_dataset(self):
        for label in os.listdir(self.path_folder):
            for img in os.listdir(self.path_folder + label):
                self.paths.append(self.path_folder + label + "/" + img)
                self.labels.append(label)
        
    def settings(self):
        self.list_dataset()
        return LabelEncoder().fit(self.labels[:])

    def decode(self, data:list):
        data = [data.item()]
        return self.encoder.inverse_transform(data)

fix_seeds(seed)
dataset = GetDataset("/kaggle/input/indian-vehicle-dataset/Vehicle_5_classes_sample/", 224)
print(dataset.decode(dataset[700][1]))
dataset.show_images()

['Tractor']


In [18]:
generator = torch.Generator()
generator.manual_seed(seed)
train_dataset, val_dataset, test_dataset = random_split(dataset, lengths=config.ratio_train_val_test, generator=generator)

In [19]:
def seed_worker(_):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=config.do_shuffle_train,
    generator=generator,
    drop_last=True,
    num_workers=4,
    pin_memory=True,
    worker_init_fn=seed_worker,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=4,
    pin_memory=True,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    drop_last=True,
)

In [29]:
import torch
from torchvision import models

model = models.resnet18()
model.load_state_dict(torch.load("/kaggle/input/vehicle/resnet18-f37072fd.pth"))
model = models.resnet18(pretrained=True)

/tmp/ipykernel_30/1226424641.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/vehicle/resnet18-f37072fd.pth"))
Downloading

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [30]:
for param in model.parameters():
    param.requires_grad = False

In [32]:
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True
model.fc = nn.Linear(512, 5)

In [33]:
trainable_params = []
for param in model.parameters():
    if param.requires_grad:
        trainable_params.append(param)


In [36]:
def accuracy(*, y_pred, y_true):
    f1 = f1_score(y_pred, y_true, average='weighted')
    acc = accuracy_score(y_pred, y_true)
    return f1, acc

In [38]:
def train_loop(config: Config,
               model: nn.Module,
               train_loader: DataLoader,
               val_loader: DataLoader,
               accuracy: tp.Callable[[torch.Tensor, torch.Tensor], tp.Tuple[float, float]],
               device: str,
               params_subset: list | None = None,
              ):
    if params_subset is None:
        params_subset = model.parameters()
    model.to(device)
    optim = Adam(params_subset, lr=config.lr)
    model.train()
    for epoch in range(config.n_epochs):
        print(f"Epoch #{epoch + 1}/#{config.n_epochs}")
        for i, (img_batch, true_labels) in enumerate(tqdm.tqdm(train_loader)):
            optim.zero_grad()
            img_batch, true_labels = img_batch.to(device), true_labels.to(device)
            loss: torch.Tensor = F.cross_entropy(model(img_batch), true_labels)
            loss.backward()
            optim.step()
        with torch.no_grad():
            model.eval()
            f1_list, acc_list = [], []
            for i, (img_batch, true_labels) in enumerate(tqdm.tqdm(val_loader)):
                pred_labels = np.stack([np.argmax(i) for i in model(img_batch).squeeze().detach().numpy()])
                print(pred_labels)
                f1, acc = accuracy(y_pred=pred_labels, y_true=true_labels)
                f1_list.append(f1)
                acc_list.append(acc)
            print(np.mean(f1_list), np.mean(acc_list))
            model.train()

In [39]:
train_loop(config, model, train_loader, val_loader, accuracy, device, params_subset=trainable_params)

Epoch #1/#100


  0%|          | 0/9 [00:00<?, ?it/s]Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
100%|██████████| 1/1 [00:11<00:00, 11.69s/it]


[4 4 1 4 2 2 2 2 2 1 1 2 2 4 4 4 1 4 2 4 1 1 1 4 2 2 2 2 2 1 4 2 2 2 2 2 2
 4 4 4 3 2 1 4 2 4 2 2 4 2 4 1 1 4 2 2 4 2 4 2 2 1 1 1]
0.2786525974025974 0.28125
Epoch #2/#100


  0%|          | 0/9 [00:00<?, ?it/s]Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
100%|██████████| 1/1 [00:10<00:00, 10.77s/it]


[4 1 4 4 3 4 2 4 1 1 1 4 2 1 2 1 1 2 2 4 0 1 4 4 1 2 1 2 1 2 4 2 2 4 1 2 1
 4 2 2 2 1 2 2 2 3 2 2 4 4 4 2 1 1 1 2 4 2 4 2 2 1 2 4]
0.3313215372355839 0.328125
Epoch #3/#100


...

[4 1 2 4 3 0 2 4 0 1 1 4 1 4 0 1 0 2 2 4 1 3 4 4 4 4 3 3 1 4 1 0 3 3 0 4 1
 0 0 4 0 4 2 4 1 0 4 2 4 4 4 0 2 1 1 1 4 4 3 3 0 1 3 3]
0.683738548273432 0.6875
Epoch #99/#100


  0%|          | 0/9 [00:00<?, ?it/s]Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
100%|██████████| 1/1 [00:11<00:00, 11.89s/it]


[4 1 2 4 3 0 0 4 0 2 1 1 1 0 0 1 3 2 0 4 1 3 4 4 4 4 3 4 1 4 1 0 0 3 0 0 2
 4 0 4 0 4 3 4 3 4 3 2 4 4 4 0 2 1 1 1 4 0 3 3 0 4 3 3]
0.5929232804232804 0.59375
Epoch #100/#100


  0%|          | 0/9 [00:00<?, ?it/s]Premature end of JPEG file
Premature end of JPEG file
Premature end of JPEG file
 11%|█         | 1/9 [00:23<03:08, 23.52s/it]Premature end of JPEG file
Premature end of JPEG file
100%|██████████| 1/1 [00:10<00:00, 10.67s/it]

[4 1 2 4 3 0 0 4 1 2 1 4 1 0 0 1 3 2 2 4 0 3 4 4 4 1 3 4 1 4 1 0 3 3 0 4 1
 0 0 4 0 4 2 4 3 0 4 1 4 4 4 0 2 1 1 1 4 0 3 3 0 4 3 3]
0.6154452907474834 0.625
